In [1]:
import numpy as np
import astropy.io.fits as pyfits
from reproject import reproject_interp

In [2]:
#### User input ####
## path to the data of interest
region_name = "W40"
data_path = "../data_CIIcorr/data_FEEDBACK/{region}/".format(region = region_name)

## first 5: Herschel, then 5: Spitzer, last: upGREAT file
name_list = ["Herschel_PACS_160_36_8", 
             "Herschel_PACS_70_36_8",
             "Herschel_SPIRE_250_36_8",
             "Herschel_SPIRE_350_36_8",
             "Herschel_SPIRE_500_36_8",
             "Spitzer_IRAC_3p6_36_8",
             "Spitzer_IRAC_4p5_36_8",
             "Spitzer_IRAC_5p8_36_8",
             "Spitzer_IRAC_8_36_8",
             "upGREAT_CII_36_8_integrated"
            ]

## target grid resolution in arcseconds
target_grid = 15.

In [3]:
for name in name_list:
    hdu = pyfits.open("{p}{reg}_{f}.fits".format(p = data_path, reg = region_name, f = name))
    header = hdu[0].header
    
    ## print warning if the image is rotated
    if(header['CROTA1'] != 0. or header['CROTA2'] != 0.):
        print('The transformation below will not be correct because there is a rotation in the image')
    
    ## calculate the ratio of new pixel resolution over old
    r_ra, r_dec = abs(target_grid/3600./header['CDELT1']), abs(target_grid/3600./header['CDELT2'])
    
    ## create the target header
    t_header = header.copy()
    
    ## update values in the target header
    t_header['CDELT1'], t_header['CDELT2'] = header['CDELT1']*r_ra, header['CDELT2']*r_dec
    t_header['NAXIS1'], t_header['NAXIS2'] = int(header['NAXIS1']/r_ra + 0.5), int(header['NAXIS2']/r_dec + 0.5)
    t_header['CRPIX1'], t_header['CRPIX2'] = header['CRPIX1']/r_ra, header['CRPIX2']/r_dec
    
    ## reproject the fits file
    array, footprint = reproject_interp(hdu, t_header)
    
    ## update file name
    new_name = name.split("_")
    if new_name[-1] == "integrated":
        new_name[-2] = str(int(target_grid))
    else:
        new_name[-1] = str(int(target_grid))
    new_name = "_".join(new_name)
    print(new_name)
    
    ## write the fits file
    new_hdu = pyfits.PrimaryHDU(array, t_header)
    new_hdu.writeto("{p}{reg}_{f}.fits".format(p = data_path, reg = region_name, f = new_name))
    

Herschel_PACS_160_36_15
Herschel_PACS_70_36_15
Herschel_SPIRE_250_36_15
Herschel_SPIRE_350_36_15
Herschel_SPIRE_500_36_15
Spitzer_IRAC_3p6_36_15
Spitzer_IRAC_4p5_36_15
Spitzer_IRAC_5p8_36_15
Spitzer_IRAC_8_36_15
upGREAT_CII_36_15_integrated
